Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 5.19 ms, sys: 193 µs, total: 5.39 ms
Wall time: 11.8 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

<Client: 'inproc://10.1.0.10/6828/1' processes=1 threads=6, memory=7.29 GB>

1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 277 ms, sys: 33.6 ms, total: 310 ms
Wall time: 2.21 s


[1.7512687970939034,
 1.6673356021151453,
 1.4889576990722637,
 1.9786912914772332,
 1.360271446351737,
 1.6121362776892811,
 1.5231517250190452,
 1.382509099840809,
 1.4524327279182094,
 1.5125590667886761,
 1.0734711579843779,
 1.3876782918105128,
 1.318207370990712,
 1.4450347895300486,
 1.3360487739000733,
 1.223552541148377,
 1.5300858505875923,
 1.4246171603769913,
 1.5085273446245466,
 1.2223676184083971,
 1.43162604838119,
 1.2755078225726466,
 1.6105012972818142,
 1.9773747259358916,
 1.6525315524440947,
 1.0862692629193091,
 1.5665387408052962,
 1.3401393075726304,
 1.4237487815055947,
 1.5875028192421305,
 1.3939880329885397,
 1.6104617096712193,
 1.5651215233271771,
 1.4749964322193496,
 1.2584166851504275,
 1.6483589956589655,
 1.5433386360186403,
 1.9068681797738667,
 1.4346533249848907,
 1.0049871694857684,
 1.2902387544543896,
 1.8425957121095107,
 1.8126350907417097,
 1.4228033620875467,
 1.9612434872933948,
 1.160955907487966,
 1.3060726953021424,
 1.5870751986124918,

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.4656369418028088,
 1.9785812053575818,
 1.5797369403624604,
 1.1948887736544673,
 1.4658462784661896,
 1.8519966268291916,
 1.4531397250557365,
 1.7311942678693972,
 1.6274760946199658,
 1.9697641687992165,
 1.6198121361640752,
 1.917000325594283,
 1.8157061366895055,
 1.588933101548861,
 1.209190563475942,
 1.0260852119991637,
 1.1254172467339523,
 1.1404638710263129,
 1.9355574218684297,
 1.6683484585133521,
 1.2658694704085482,
 1.0445689480402356,
 1.4961317919808148,
 1.868285954858894,
 1.7683001175521826,
 1.463930586292761,
 1.99026364230017,
 1.583353790621933,
 1.2603949479104686,
 1.7525116760557533,
 1.8499286002831885,
 1.0318340743421368,
 1.6976900943324196,
 1.5925473876173086,
 1.859946911251538,
 1.857008524656765,
 1.1914818636396336,
 1.5696734373394432,
 1.4214351773072316,
 1.3649036978715352,
 1.0885241725562071,
 1.5803962616923735,
 1.5715105984126032,
 1.2724763066443507,
 1.2321635834371425,
 1.7509194855078731,
 1.6757744586036347,
 1.5415369730468143,
 1

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 214 ms, sys: 32.5 ms, total: 247 ms
Wall time: 2.21 s
